# Getting Started With Exploratory Data Analysis (EDA)

<a href="https://colab.research.google.com/github/BU-Spark/ml-549-course/blob/main/phase3_EDA/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook serves as a starter guide or template for exploratory data analysis. It will go over the topics mentioned in the EDA guide. 

In [ ]:
# let's start off by importing the libraries we will need for eda
import pandas as pd
import numpy as np 

# for visualizations : 
import seaborn as sns
import matplotlib.pyplot as plt

The dataset we will be using in this tutorial is from Analyze Boston. Analyze Boston is the City of Boston's data hub and is a great resource for data sets regarding the city. 

We will be working with the 2022 311 Service Requests dataset. The dataset consists of service requests from all channels of engagement. 311 allows you to report non-emergency issues or request non-emergency City services. 

Link: https://data.boston.gov/dataset/311-service-requests 

In [ ]:
import os
import requests
from tqdm import tqdm

def download_csv(url, filename):
    # Check if the file already exists
    if os.path.exists(filename):
        print(f"The file {filename} already exists.")
        return

    response = requests.get(url, stream=True)

    total_size_in_bytes= int(response.headers.get('content-length', 0))
    block_size = 1024 #1 Kibibyte
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

    with open(filename, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("ERROR, something went wrong")

# Use the function
url = "https://data.boston.gov/datastore/dump/81a7b022-f8fc-4da5-80e4-b160058ca207?bom=True"
filename = "311-requests.csv"
download_csv(url, filename)

In [ ]:
# read in dataset
df = pd.read_csv('311-requests.csv') 

In [ ]:
# let's look at the first five rows of the dataset
df.head()

How many observations/rows are there? <br>

How many variables/columns are there? <br>

What kinds of variables are there? Qualitative? Quantitative? Both? <br>

In [ ]:
# number of observations 
df.shape[0]

In [ ]:
# to see column name, count, and dtype of each column 
df.info()

There are 146373 rows (observations). <br>

There are 29 columns (variables). <br> 

There are both categorical and numerical variables. At quick glance there seems to be more categorical variables than numerical variables. 

Categorical Variables:
```case_status```, ```neighborhood```, ```source```, etc. 

Numerical Variables: 
... maybe not?

The ```case_enquiry_id``` is a unique identifier for each row, ```closedphoto``` has 0 non-null values so it might be worth it to drop this column since there is no additional information we can gather, columns such as ```location_zipcode```, ```latitude```, ```longitude``` not exactly numeric varaibles, since they are numbers that represent different codes. 

### Cleaning 

Let's convert the three time variables (```open_dt```, ```target_dt```, and ```closed_dt```) from objects to pandas datetime objects.
Let's focus on service requests for a set period of time in 2022. 
We will start by filtering for service requests that were opened from January 2022 to March 2022. 

In [ ]:
# changing the three columns with dates and times to pandas datetime object 
df['open_dt'] = pd.to_datetime(df['open_dt'])
df['sla_target_dt'] = pd.to_datetime(df['sla_target_dt'])
df['closed_dt'] = pd.to_datetime(df['closed_dt'])

# output is long, but run the line below to check the type of the three columns 
df.dtypes

In [ ]:
# filter data for 311 requests from january 2022 to march 2022 
df_filtered = df.loc[(df['open_dt'] >= '2022-01-01') &
                  (df['open_dt'] < '2022-03-31')]
df_filtered.head()

From our previous observation, since ```closedphoto``` column does not contain any non-null values, let's drop it. 

In [ ]:
# drop closedphoto column
df_filtered = df_filtered.drop(columns=['closed_photo'])

After filtering the service requests, let's see how many observations we are left with.  

In [ ]:
# how many requests were opened from Jan 2022 to March 2022
df_filtered.shape[0]

From a quick preview of the dataframe, we can see that some of the requests are still open. 
Let's see how many observations are open vs. closed and then how many are ontime vs. overdue from the set of requests from January 2022 to March 2022. 

In [ ]:
# checking how many open vs. closed cases
df_filtered['case_status'].value_counts()

In [ ]:
# visualize case_status in pie chart, set color palette 
colors = sns.color_palette('muted')[0:5]
ax = df_filtered['case_status'].value_counts().plot.pie(colors=colors)
ax.yaxis.set_visible(False)

In [ ]:
# checking how many ontime vs. overdue cases 
df_filtered['on_time'].value_counts()

In [ ]:
# visualize ontime in pie chart, set color palette 
colors = sns.color_palette('bright')[0:5]
ax = df_filtered['on_time'].value_counts().plot.pie(colors=colors)
ax.yaxis.set_visible(False)

### Descriptive Statistics 

Pandas makes this easy! We can use ```describe()``` to get the descriptive statistics of the numerical columns. 

In [ ]:
df_filtered.describe()

As mentioned before, the ```case_enquiry_id```, ```location_zipcode```, ```latitude```, and ```longitude``` columns are not numeric variables. The descriptive statistics are not very useful in this situation. 

What would be a useful numeric variable is the duration of a request. Let's calculate the duration of each of the requests from January 2022 to March 2022 and add it as a new column in our dataframe. 

In [ ]:
# calculating case duration and adding a new column (case_duration) to the dataframe 
duration = df_filtered['closed_dt'] - df_filtered['open_dt']
df_filtered = df_filtered.assign(case_duration=duration)
df_filtered.head()

Now we can see the new ```case_duration``` column. Some values are ```NaT```, which means there is a missing date. This makes sense because the ```case_status``` is ```OPEN```. 

Let's filter out the open cases and focus on analyzing the duration of the closed cases. 

In [ ]:
# filter out the open cases
df_closed = df_filtered.loc[(df_filtered['case_status'] == "Closed")]
df_closed.head()

With the closed cases, let's calculate the descriptive statistics of the new ```case_duration``` column. 

In [ ]:
# let's calculate the descriptive statistics again 
# using double brackets to display in a *fancy* table format
df_closed[['case_duration']].describe()

From the table, we can see that the average case duration is ~4.5 days. <br>
The standard deviation for the case duration is ~15.4 days. <br>
The minimum time a case takes to close is 4 minutes. <br>
The maximum time a case takes to close is ~181.6 days. <br>
The inter-quartile range (IQR) is the difference between the 25% and 75% quantiles. <br>


We can also calculate the *mode* and *median*.

In [ ]:
df_closed['case_duration'].mode()

In [ ]:
df_closed['case_duration'].median()

The descriptive statistics summary in table form is nice, but it would be nice to visualize the data in a histogram. Simply trying to plot using the values in the ```case_duration``` column will case an error. 

Currently, the values in ```case_duration``` are of type ```timedelta64[ns]```, ```df_closed['case_duration']``` is a Timedelta Series. We will need to apply what is called a frequency conversion to the values. 

"Timedelta Series, TimedeltaIndex, and Timedelta scalars can be converted to other 'frequences' by dividing by another timedelta, or by astyping to a specific timedelta type." (See the link below for more information and code examples!)

https://pandas.pydata.org/pandas-docs/stable/user_guide/timedeltas.html


In [ ]:
# dividing the case_duration values by Timedelta of 1 day 
duration_days = ( df_closed['case_duration'] / pd.Timedelta(days=1))

# adding calculation to dataframe under duration_in_days column 
df_closed = df_closed.assign(duration_in_days=duration_days)

# display descriptive statistics summary with new column addition 
df_closed[['duration_in_days']].describe()

In [ ]:
# using seaborn library for visualizations 
sns.set_theme() # use this if you dont want the visualizations to be default matplotlibstyle
sns.displot(df_closed, x="duration_in_days", binwidth=1)

From the plot above, the data seems to be skewed right meaning the right tail is much longer than the left. Let's try playing with different bin widths. 

In [ ]:
# trying different bin sizes 
sns.displot(df_closed, x="duration_in_days", binwidth=5)

In [ ]:
# trying different bin sizes
sns.displot(df_closed, x="duration_in_days", binwidth=25)

Since the data is heavily skewed, let's apply log transformation to the data. The log transformation will *hopefully* reduce or remove the skewness of the original data. The assumption is that the original data follows a log-normal distribution. 

In [ ]:
# log-scale transformation since the data is heavliy skewed 
# add bin_width parameter to change bin sizes
sns.displot(df_closed, x="duration_in_days", log_scale=True)

### Which neighborhoods had the most requests from January 2022 - March 2022? 

To answer this question, we will take a look at the ```neighborhood``` column. 

In [ ]:
# has 25 unique values so a pie chart probably is not the best option
len(df_closed['neighborhood'].unique()) 

In [ ]:
# plot neighborhood counts 
sns.countplot(x="neighborhood", data=df_closed).set_title('Number of Requests by Neighborhood')

Yikes! The x-axis labels are pretty hard to read. Let's fix that by plotting the bars horizontally. 

In [ ]:
# fixing orientation of the labels 
sns.countplot(y="neighborhood", data=df_closed).set_title('Number of Requests by Neighborhood')

From the plot we can see that Dorchester has the most requests, followed by South Boston/South Boston Waterfront, then Roxbury. There's a bar that doesn't have a name...that's strange. Let's display the exact counts for each neighborhood. 

In [ ]:
# displaying number of requests by neighborhood in table form 
df_closed['neighborhood'].value_counts()

There are 476 requests without a neighborhood label. 

In [ ]:
# uncomment and run the line below to check for the empty neighborhood label 
# print(df_closed['neighborhood'].unique())

# gather the rows where neighborhood == ' ' 
df_no_neighborhood = df_closed.loc[(df_closed['neighborhood'] == ' ')]
df_no_neighborhood.head(15) # display first 15 rows

In [ ]:
print(df_no_neighborhood['latitude'].unique())
print(df_no_neighborhood['longitude'].unique())

The latitude and longitude values are the same for all of the rows without a ```neighborhood``` value. We can use the *Geopy* module to convert the latitude and longitude coordinates to a place or location address - also referred to as reverse geocoding. 

In [ ]:
# import geopy 
from geopy.geocoders import Nominatim 

# make a Nominatim object and initialize, specify a user_agent 
# Nominatim requires this value to be set to your application name, to be able to limit the number of requests per application
# Nominatim is a free service but provides low request limits: https://operations.osmfoundation.org/policies/nominatim/
geolocator = Nominatim(user_agent="eda_geotest")

# set latitude and longitude and convert to string 
lat = str(df_no_neighborhood['latitude'].unique()[0])
long = str(df_no_neighborhood['longitude'].unique()[0])

# get the location information
location = geolocator.reverse(lat + "," +long)

# display location information, add .raw for more details
print(location.raw)

Quick Google Maps search of the location confirms that (42.3594, -71.0587) is Government Center. The output from *geopy* is Sear's Crescent and Sears' Block which are a pair of buildings adjacent to City Hall and City Hall Plaza, Government Center.

Another quick look at the output from *geopy* shows that the ```lat``` and ```lon``` values are similar but different from the latitude and longitude values in the dataset. 

The requests without a neighborhood value have a general location of Government Center. At least we can confirm that requests without a neighborhood value are not outside of Boston or erroneous. 


### During January 2022 - March 2022, where did the most case requests come from?

To answer this question, we will take a look at the ```source``` column. 

In [ ]:
# has only 5 unique values so in this case we can use a pie chart 
len(df_closed['source'].unique())

In [ ]:
# displaying the number of requests by each source type 
df_closed['source'].value_counts()

In [ ]:
# visualizing the breakdown of where case requests come from 
# seaborn doesn't have a default pie chart but you can add seaborn color palettes to matplotlib plots

colors = sns.color_palette('pastel')[0:5]
ax = df_closed['source'].value_counts().plot.pie(colors=colors)

In [ ]:
# label each slice with the percentage of requests per source 
ax = df_closed['source'].value_counts().plot.pie(colors=colors,autopct='%1.1f%%')

# run the following to remove the default column name label *source*
#ax.yaxis.set_visible(False)

From the pie chart, 54% of the requests from January 2022 - March 2022 came from the Citizens Connect App, 35.4% came from a Constituent Call, followed by 6.4% from the City Worker App. 

### How many different types of requests were there from January 2022 - March 2022?

To answer this question, we will take a look at the ```reason``` column. 

In [ ]:
# how many different reasons are there 
len(df_closed['reason'].unique())

In [ ]:
# number of requests by reason 
df_closed['reason'].value_counts()

There were 38 different types of requests from January 2022 - March 2022, the top three with most requests being *Enforcement & Abandoned Vehicles* with 14,908 requests, *Code Enforcement* with 10,437 requests, then *Street Cleaning* with 8,477 requests. 

In [ ]:
# top case request reason by neighborhood 
df_closed.groupby(['neighborhood'])['reason'].describe()

In [ ]:
# get counts for each request reason by neighborhood 
reason_by_neighborhood = df_closed.groupby(['neighborhood', 'reason'])['duration_in_days'].describe()[['count']]
reason_by_neighborhood

In [ ]:
# run this cell to write the reason by neighborhood to a csv to see all rows of data 
reason_by_neighborhood.to_csv('reasons_by_neighborhood.csv')

In [ ]:
# let's take a look at the South End neighborhood specifically 
south_end_df = df_closed.loc[(df_closed['neighborhood'] == 'South End')]
south_end_df.groupby(['reason'])['duration_in_days'].describe()[['count']]

### What types of cases typically take the longest to resolve?

To answer this question, let's take a look at the ```duration_in_days``` and ```reason``` columns.

In [ ]:
# what types of cases typically take the longest 
# case_duration by reason 

sns.catplot(x="reason", y="duration_in_days", kind="box", data=df_closed,)

In [ ]:
# The chart is kind of difficult to read... 
# Let's fix the size of the chart and flip the labels on the x-axis 

sns.catplot(y="reason", x="duration_in_days", kind="box", data=df_closed,
            height = 8, aspect = 1.25)

Box plots display the five-number-summary, which includes: the minimum, the maximum, the sample median, and the first and third quartiles. 

The box plot shows the distribution ```duration_in_days``` in a way that allows comparisions between case ```reasons```. Box plots show the distribution of a numerical variable broken down by a categorical variable. 

The box shows the quartiles of the ```duration_in_days``` and the whiskers extend to show the rest of the distribution (minimum and maximum). Points that are shown outside of the whiskers are determined to be *outliers*. The line inside the box is the median. 

In [ ]:
# descriptive statistics for duration_in_days by case reason 
# box plot in table form 
df_closed.groupby(['reason'])['duration_in_days'].describe()

Graffiti cases take on average take the longests time to resolve, 60.796 days. 

Do cases typically take longer in one neighborhood over another?

In [ ]:
# do cases typically take longer in one neighborhood over another?

sns.catplot(y="neighborhood", x="duration_in_days", kind="box", data=df_closed,
            height = 8, aspect = 1.25)

The box plot above shows several outliers for each category (```neighborhood```) making it difficult to read and quite overwhelming. 

Let's display the information in table form. 

In [ ]:
# in table form 
df_closed.groupby(['neighborhood'])['duration_in_days'].describe()

In January 2022 - March 2022, cases took the longest in Chestnut Hill. Cases typically lasted on average 19.075 days but there were only 4 cases located in Chestnut Hill during this time. Smaller sample sizes could mean more variability (look at standard deviation to explain the spread of observations).  

We can further look at the population of Chestnut Hill versus the other neighborhoods to try and make sense of this low case count. Additionally, we can broaden the time period of the cases to see if Chestnut Hill still has a low case count. 

From the table above we can see how long cases take by each neighborhood, it would be interesting to further breakdown by case reason for each neighborhood. 

### Wrap Up, Next Steps 

Further analysis could be done using the 311 dataset. Using the 311 data from previous years, we can see how number of requests have changed over the years, or how case duration may have changed over the years. 

Since most requests have latitude and longitude coordinates it could be interesting to plot each case request on a map to see if there are clusters of requests in certain locations.  

Next steps could include gathering demographic data to overlay on top of the 311 dataset for further analysis. Another possible next step would be to build a model to predict how long a request could take given the request reason, subject, location, source, etc. 